In [73]:
import numpy as np

good_patterns = [[1, 0, 0, 1]]
training_data_length = 1000

def getTrainingData(length, pattern_length):
    training_data = np.array([np.array([0, 1]).repeat(pattern_length / 2)] * length)
    
    for pattern in training_data:
        np.random.shuffle(pattern)   
    
    return training_data

def getCorrectResults(patterns):
    return np.array([1 if list(pattern) in good_patterns else 0 for pattern in patterns])

training_data = getTrainingData(training_data_length, pattern_length=4)
correct_results_training_data = getCorrectResults(training_data)

In [74]:
import tensorflow.keras as keras


NN = keras.Sequential()
NN.add(keras.Input(shape=(4,)))
NN.add(keras.layers.Dense(1, activation="sigmoid")) # sigmoid(x) = 1 / (1 + exp(-x)) here

NN.compile(loss="binary_crossentropy", optimizer="Adam", metrics=["accuracy"])

NN.fit(training_data, correct_results_training_data, epochs=150)

NN.summary()

weights = NN.get_weights()

# print the weights
for i, layer_weights in enumerate(weights):
    print(f"Layer {i}:")
    for j, weight in enumerate(layer_weights):
        print(f"Weight {j}: {weight}")

Epoch 1/300
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 508us/step - accuracy: 0.6798 - loss: 0.6514 
Epoch 2/300
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 518us/step - accuracy: 0.6601 - loss: 0.6405
Epoch 3/300
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 504us/step - accuracy: 0.6765 - loss: 0.6152
Epoch 4/300
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 468us/step - accuracy: 0.8349 - loss: 0.6210
Epoch 5/300
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 353us/step - accuracy: 0.8428 - loss: 0.5986
Epoch 6/300
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.8628 - loss: 0.5499 
Epoch 7/300
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 515us/step - accuracy: 0.8392 - loss: 0.5548
Epoch 8/300
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 543us/step - accuracy: 0.8478 - loss: 0.5391
Epoch 9/300
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 419us/step - accuracy: 0.8483 - loss: 0.5325
Epoch 10/300
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.8431 - loss: 0.5168 
Epoch 11/300
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 528us/step - accuracy: 0.8461 - loss: 0.5116
Epoch 12/300
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s

Model: "sequential_12"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense_51 (Dense)                │ (None, 1)              │             5 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 17 (72.00 B)

 Trainable params: 5 (20.00 B)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 12 (52.00 B)

Layer 0:
Weight 0: [3.1264513]
Weight 1: [-5.7158365]
Weight 2: [-5.0349107]
Weight 3: [3.1594713]
Layer 1:
Weight 0: -2.677578926086426


Now we want to test the Network with a  new list of patterns.

In [75]:
test_data = np.copy(training_data)
np.random.shuffle(test_data)

# NN.predict returns an array with shape=(trainging_data_length = 1000, output_neurons = 1) 
# so we flatten it to get a 1D array with our predictions
results_on_test_data = NN.predict(test_data).flatten() 
correct_results_test_data = getCorrectResults(test_data)

all_correct = np.array_equal(correct_results_test_data, np.round(results_on_test_data))
max_difference = max(abs(results_on_test_data - correct_results_test_data))

print(f"all test data correct, when rounded: {all_correct}")
print(f"maximum difference from correct result: {max_difference}")

32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 777us/step
all test data correct, when rounded: True
maximum difference from correct result: 0.026381850242614746


In [174]:
import numpy as np

training_data_length = 1000

good_patterns = [
    list(np.array(list(zip(np.ones(32), np.zeros(32)))).flatten()),
    list(np.array(list(zip(np.zeros(32), np.ones(32)))).flatten())
]

def getTrainingData(length, pattern_length):
    training_data = np.array([np.array([0, 1]).repeat(pattern_length / 2)] * length)
    
    for i, pattern in enumerate(training_data):
        # this time we additionally inject some good patterns, 
        # because otherwise there wouldnt be enough good patterns in training_data
        rand = np.random.rand()
        if rand < 0.2:
            training_data[i] = np.array(good_patterns[0], dtype=int)
        elif rand < 0.4:
            training_data[i] = np.array(good_patterns[0], dtype=int)
        else: 
            np.random.shuffle(pattern) # shuffle in place, pattern is still the actual element from the training_data array
    
    return training_data

training_data = getTrainingData(training_data_length, pattern_length=64)
correct_results_training_data = getCorrectResults(training_data)

[[1 1 1 ... 0 0 1]
 [0 0 0 ... 0 1 1]
 [0 0 1 ... 0 1 1]
 ...
 [1 0 1 ... 0 1 0]
 [1 1 1 ... 1 1 1]
 [1 0 0 ... 1 1 1]]


In [184]:
NN = keras.Sequential()
NN.add(keras.Input(shape=(64,)))
NN.add(keras.layers.Dense(32, activation="relu"))
NN.add(keras.layers.Dense(16, activation="relu"))
NN.add(keras.layers.Dense(8, activation="relu"))
NN.add(keras.layers.Dense(4, activation="relu"))
NN.add(keras.layers.Dense(2, activation="relu"))
NN.add(keras.layers.Dense(1, activation="sigmoid")) # sigmoid(x) = 1 / (1 + exp(-x)) here

NN.compile(loss="binary_crossentropy", optimizer="Adam", metrics=["accuracy"])

NN.fit(training_data, correct_results_training_data, epochs=300)

NN.summary()

Epoch 1/300
32/32 ━━━━━━━━━━━━━━━━━━━━ 1s 948us/step - accuracy: 0.4845 - loss: 0.6924 
Epoch 2/300
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 861us/step - accuracy: 0.7333 - loss: 0.6596
Epoch 3/300
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 513us/step - accuracy: 1.0000 - loss: 0.3857
Epoch 4/300
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 548us/step - accuracy: 1.0000 - loss: 0.2039
Epoch 5/300
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 812us/step - accuracy: 1.0000 - loss: 0.0815
Epoch 6/300
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 1.0000 - loss: 0.0197 
Epoch 7/300
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 437us/step - accuracy: 1.0000 - loss: 0.0067
Epoch 8/300
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 616us/step - accuracy: 1.0000 - loss: 0.0036
Epoch 9/300
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 843us/step - accuracy: 1.0000 - loss: 0.0022
Epoch 10/300
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 513us/step - accuracy: 1.0000 - loss: 0.0016
Epoch 11/300
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 1.0000 - loss: 0.0012 
Epoch 12/300
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s

Model: "sequential_20"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense_67 (Dense)                │ (None, 32)             │         2,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_68 (Dense)                │ (None, 16)             │           528 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_69 (Dense)                │ (None, 8)              │           136 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_70 (Dense)                │ (None, 4)              │            36 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_71 (Dense)                │ (None, 2)              │            10 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_72 (Dense)                │ (None, 1)              │             3 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 8,381 (32.74 KB)

 Trainable params: 2,793 (10.91 KB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 5,588 (21.83 KB)

In [186]:
test_data = getTrainingData(500_000, pattern_length=64)

results_on_test_data = NN.predict(test_data).flatten()
correct_results_test_data = getCorrectResults(test_data)

all_correct = np.array_equal(correct_results_test_data, np.round(results_on_test_data))
max_difference = max(abs(results_on_test_data - correct_results_test_data))

print(f"all test data correct, when rounded: {all_correct}")
print(f"maximum difference from correct result: {max_difference}")

15625/15625 ━━━━━━━━━━━━━━━━━━━━ 7s 470us/step
all test data correct, when rounded: True
maximum difference from correct result: 0.09910716116428375
